In [1]:
from random import randrange,randint
import pandas as pd

In [2]:
def gen_axiom(maxlen=3):
    borel_field = 'MIU'
    return ''.join([borel_field[randrange(0,3)] for i in range(maxlen)])

In [3]:
def rule1(s):
    if s[-1] == 'I':
        return s + 'U'
    return s

def rule2(s):
    if s[0] == 'M':
        return 'M' + s[1:] + s[1:]
    return s

def rule3(s):
    if "III" in s:
        return s.replace("III","U")
    else:
        return s
    
def rule4(s):
    if "UU" in s:
        return s.replace("UU","")
    return s

In [4]:
def gen_derivation(axiom,derivations=10,rules=[rule1,rule2,rule3,rule4]):
    theorem = []
    for i in range(derivations):
        func = rules[randrange(0,4)]
        derivation = func(axiom)
        if derivation == '':
            theorem.append(axiom)
            return theorem,len(theorem)
        axiom = derivation
        theorem.append(axiom)
    return theorem,len(theorem)

In [5]:
def get_first_cliff_and_drop(diff_lens,diffs):
    cliff_drops = {
        'drop_ixs':[],
        'cliffs':[],
        'drops':[],
        'cliff_drop_diff':[],
        'pct_cliff_drop_diff':[],
        'overall_change':[]
    }
     
    for ix,val in enumerate(diffs):
        if (val<0) and (diffs[ix+1] >= 0):
            cliff_drops['overall_change'].append(sum(diffs))
            cliff_drops['drop_ixs'].append(ix+1)
            cliff_drops['cliffs'].append(diff_lens[ix])
            cliff_drops['drops'].append(diff_lens[ix+1])
            
            cliff_drop_diff = diff_lens[ix]-diff_lens[ix+1]
            cliff_drops['cliff_drop_diff'].append(cliff_drop_diff)
            cliff_drops['pct_cliff_drop_diff'].append(cliff_drop_diff/diff_lens[ix])
    return cliff_drops

In [6]:
def create_random_axiom_dataset(nsamples=20,max_axiom_size=3,max_derivations=20):
    sample_data = []
    all_derivations = []
    for i in range(nsamples):
        axiom = gen_axiom(maxlen=randint(2,max_axiom_size))
        derivations,derivation_len = gen_derivation(axiom,max_derivations)
        diff_lens = [len(step) for step in derivations]
        diffs = [diff_lens[i] - diff_lens[i-1] for i in range(len(diff_lens))][1:]

        try:
            metadata = pd.DataFrame(get_first_cliff_and_drop(diff_lens,diffs))
            metadata.loc[:,'axiom'] = axiom
            metadata.loc[:,'axiom_len'] = len(axiom)
            metadata.loc[:,'sample_id'] = str(i)
            metadata.loc[:,'derivation_length'] = derivation_len
            metadata.loc[:,'step_longest'] = max(diff_lens)
            metadata.loc[:,'step_shortest'] = min(diff_lens)
            metadata.loc[:,'MI_ind'] = 'MI' in derivations
            # location of MI
            if 'MI' 
            sample_data.append(metadata)
        except:
            continue
        all_derivations.append(derivations)
    
    sample_data = pd.concat(sample_data)
    pct_unique_id = len(sample_data['sample_id'].unique())/nsamples
    return sample_data,all_derivations,pct_unique_id

In [7]:
nsamp = 100000
random_theorems,derivations,pct_samples =\
    create_random_axiom_dataset(nsamples=nsamp,max_axiom_size=5,max_derivations=20)
pct_samples

0.24102

In [8]:
random_theorems.to_clipboard(index=False)

In [9]:
# get length of the longest string
# get length of the shortest string

In [10]:
random_theorems[random_theorems['MI_ind']==True]

,cliff_drop_diff,cliffs,drop_ixs,drops,overall_change,pct_cliff_drop_diff,axiom,axiom_len,sample_id,derivation_length,step_longest,step_shortest,MI_ind
0,2,4,1,2,189,0.500000,MIUU,4.0,22,20.0,193.0,2.0,True
0,2,5,5,3,19,0.400000,MUUI,4.0,477,20.0,21.0,2.0,True
0,2,4,1,2,9,0.500000,MUUI,4.0,506,20.0,13.0,2.0,True
0,20,33,8,13,11,0.606061,MI,2.0,909,20.0,51.0,2.0,True
1,4,11,18,7,11,0.363636,MI,2.0,909,20.0,51.0,2.0,True
0,2,4,5,2,15,0.500000,UUMI,4.0,1440,20.0,19.0,2.0,True
1,2,5,8,3,15,0.400000,UUMI,4.0,1440,20.0,19.0,2.0,True
0,42,65,13,23,5,0.646154,MI,2.0,2347,20.0,65.0,2.0,True
1,20,24,18,4,5,0.833333,MI,2.0,2347,20.0,65.0,2.0,True
0,8,21,6,13,11,0.380952,MI,2.0,2504,20.0,25.0,2.0,True
